# General EDA - preparation notebook
* Run the scripts in the folder `run_first_preprocessing` first.
* EDA markers in this notebook are based on the original astro masks.
* This notebook and the following (`EDA_figures`) does not tie into the segmentation modeling part of this project but rather offers a general overview of the image properties.


## Marker generation for EDA

### Based on the greyscale values:
* Number of channels (RGB vs grey-scale)
* Mean and std values per channel (are some images brighter or have a larger dynamical range?)
* Number of pixels per annotations (how large are the cells?)
* Number of annotations per image (how many cells are there?)
* Number of pixels across all annotations per image (how much of the image is covered with the objects of interest?)

### Based on the annotation masks:
* Number of annotations
* Area covered by a single annotation
* Area covered by all annotations
* Min area covered by a single annoation
* Max area covered by a single annotation
* Mean area covered by a single annotation


### Load the data and preallocate space for the new variables

In [ ]:
import os
import pathlib

import cv2 as cv

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from IPython.core.magic import register_cell_magic

In [ ]:
# Specify the project directory
curr_dir = os.getcwd()
parent_dir = pathlib.Path(curr_dir).parents[1]

# Read meta data
df = pd.read_csv(f"{parent_dir}/data/data_original/train.csv")

In [ ]:
# New markers
df["n_channels"] = 0

df["min_grey_global"]  = -99
df["max_grey_global"]  = -99
df["mean_grey_global"] = -99
df["std_grey_global"]  = -99

df["min_grey_cell"]    = -99
df["max_grey_cell"]    = -99
df["mean_grey_cell"]   = -99
df["std_grey_cell"]    = -99
df["min_grey_nocell"]  = -99
df["max_grey_nocell"]  = -99
df["mean_grey_nocell"] = -99
df["std_grey_nocell"]  = -99

df["n_annotations"]                = -99
df["perc_total_annotation"]        = -99
df["perc_single_annotation"]       = -99
df["min_perc_single_annotation"]   = -99
df["max_perc_single_annotation"]   = -99
df["range_perc_single_annotation"] = -99
df["mean_perc_single_annotation"]  = -99

### Greyscale markers

In [ ]:
id_prev = "no_prev_id"

for idx in df.index:
    img_id = df.loc[idx,"id"]
    
    # Operations that are only executed once per image not once per df row
    if id_prev != img_id:
        img = cv.imread(f"{parent_dir}/data/data_original/train/{img_id}.png")
        b, g, r = cv.split(img)
        
        # Number of channels
        if np.array_equal(b,g) and np.array_equal(b,r): # all channels are identical
            df.loc[df["id"]==img_id,"n_channels"] = 1
            img = img[:,:,0] # only select one channel
        else:
            df.loc[df["id"]==img_id,"n_channels"] = 3
        
        # Global grey scale infos
        df.loc[df["id"]==img_id,"min_grey_global"]  = np.min(img) 
        df.loc[df["id"]==img_id,"max_grey_global"]  = np.max(img)
        df.loc[df["id"]==img_id,"mean_grey_global"] = np.mean(img)
        df.loc[df["id"]==img_id,"std_grey_global"]  = np.std(img)
        
        # Mask-dependent greyscale infos
        msk = cv.imread(f"{parent_dir}/data/data_preprocessed/mask_groundtruth/{img_id}_mask.png")
        msk = msk[:,:,0]

        # Mask==1 grey scale infos
        df.loc[df["id"]==img_id,"min_grey_cell"]   = img[msk>0].min()
        df.loc[df["id"]==img_id,"max_grey_cell"]   = img[msk>0].max()
        df.loc[df["id"]==img_id,"mean_grey_cell"]  = img[msk>0].mean()
        df.loc[df["id"]==img_id,"std_grey_cell"]   = img[msk>0].std()

        # Mask==0 grey scale infos
        df.loc[df["id"]==img_id,"min_grey_nocell"]   = img[msk==0].min()
        df.loc[df["id"]==img_id,"max_grey_nocell"]   = img[msk==0].max()
        df.loc[df["id"]==img_id,"mean_grey_nocell"]  = img[msk==0].mean()
        df.loc[df["id"]==img_id,"std_grey_nocell"]   = img[msk==0].std()

    id_prev = img_id

### Annotation markers

In [ ]:
id_prev = "no_prev_id"

for idx in df.index:
    img_id = df.loc[idx,"id"]
    
    # Operations carried out per single annotation
    # Create a temporary mask for a single annotation
    msk_tmp = np.zeros(msk.shape[0] * msk.shape[1])
    annotation = df.loc[idx,"annotation"].split()
    list_1s = [(int(start)-1, int(start)-1 + int(length)) for start, length in zip(annotation[0:][::2], annotation[1:][::2])]
    for start, end in list_1s:
        msk_tmp[start:end] += 1
    
    df.loc[idx,"perc_single_annotation"] = np.sum(msk_tmp>0) / (msk.shape[0] * msk.shape[1])
    
    # Operations that are only executed once per image not once per df row 
    if id_prev != img_id:   
        msk = cv.imread(f"{parent_dir}/data/data_preprocessed/mask_groundtruth/{img_id}_mask.png")
        msk = msk[:,:,0]
        
        df.loc[df["id"]==img_id,"n_annotations"]           = df[df["id"]==img_id].shape[0]
        df.loc[df["id"]==img_id,"perc_total_annotation"]   = np.sum(msk>0) / (msk.shape[0] * msk.shape[1])
        
        # put first value for min/max here
        df.loc[df["id"]==img_id,"min_perc_single_annotation"] = df.loc[idx,"perc_single_annotation"]
        df.loc[df["id"]==img_id,"max_perc_single_annotation"] = df.loc[idx,"perc_single_annotation"]
    
    else: # If we're still looping inside the same image, we do the checks for min/max annotation size
        if df.loc[idx,"perc_single_annotation"] < df.loc[idx,"min_perc_single_annotation"]:
            df.loc[df["id"]==img_id,"min_perc_single_annotation"] = df.loc[idx,"perc_single_annotation"]
            
        if df.loc[idx,"perc_single_annotation"] > df.loc[idx,"max_perc_single_annotation"]:
            df.loc[df["id"]==img_id,"max_perc_single_annotation"] = df.loc[idx,"perc_single_annotation"]
          
    id_prev = img_id

In [ ]:
df["mean_perc_single_annotation"] = df.groupby("id")["perc_single_annotation"].transform(np.mean)

df["range_perc_single_annotation"] = df["max_perc_single_annotation"] - df["min_perc_single_annotation"]

### Store the results for future use

In [ ]:
df.to_csv("eda_markers.csv", index=False)

## Figure functions
Define some functions used to display the eda markers in the next notebook


### For storing the functions as python script

In [ ]:
!rm -f figure_functions.py

@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
        print("Appended to file ", file)
    else:
        print('Written to file:', file)
    with open(file, mode) as f:
        f.write(cell.format(**globals()))        
    get_ipython().run_cell(cell)

In [ ]:
%%write_and_run figure_functions.py
# Imports
import pandas as pd
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

import skimage
from skimage import io
from skimage import color
from skimage import filters

from sklearn.metrics import classification_report, confusion_matrix
import itertools

import os
import pathlib



In [ ]:
%%write_and_run -a figure_functions.py
# Get the ID of the images closest to the center of a bivariate feature distribution (per class)
# and return the dataframe with an additional column indicating the corresponding rows
def get_prototypical_id(df, class_names, feature_1, feature_2):

    df["center_img"] = 0
    
    for class_name in class_names:
        df2 = df[df["cell_type"]==class_name].copy()

        feature_1_mean = np.mean(df2[feature_1])
        feature_2_mean = np.mean(df2[feature_2])

        df2["eucl_dist"] = np.sqrt(
                                np.power(df2[feature_1]-feature_1_mean, 2) + 
                                np.power(df2[feature_2]-feature_2_mean, 2))

        min_dist = np.min(abs(df2["eucl_dist"]))

        img_id = df2.loc[abs(df2["eucl_dist"])==min_dist]["id"].values[0]
        
        df.loc[df["id"]==img_id,"center_img"] = 1
        
    return df



def axlim(df, feature_1, feature_2):
    xmin_tmp = np.min(df[feature_1])
    xmax_tmp = np.max(df[feature_1])

    xmin = xmin_tmp - 0.05 * abs(xmax_tmp - xmin_tmp)
    xmax = xmax_tmp + 0.05 * abs(xmax_tmp - xmin_tmp)

    ymin_tmp = np.min(df[feature_2])
    ymax_tmp = np.max(df[feature_2])

    ymin = ymin_tmp - 0.05 * abs(ymax_tmp - ymin_tmp)
    ymax = ymax_tmp + 0.05 * abs(ymax_tmp - ymin_tmp)
    
    return (xmin, xmax), (ymin, ymax)



# Function that plots a bivariate feature distribution and the most prototypical images per class
def typical_img(df, feature_1, feature_2, show_unity_line = False, log_axes = False):
    
    # Specify the project directory for loading the images
    curr_dir = os.getcwd()
    parent_dir = pathlib.Path(curr_dir).parents[1]
    
    # Get the class names
    class_names = sorted(df["cell_type"].unique().tolist())

    # Instantiate figure and create subplot grid
    fig, ax = plt.subplots(8)
    fig.set_figheight(8)
    fig.set_figwidth(16)

    ax[0] = plt.subplot2grid(shape=(10, 7), loc=(1, 0), rowspan=8, colspan=3)
    ax[1] = plt.subplot2grid(shape=(10, 7), loc=(0, 3), rowspan=5, colspan=2)
    ax[2] = plt.subplot2grid(shape=(10, 7), loc=(0, 5), rowspan=5, colspan=2)
    ax[3] = plt.subplot2grid(shape=(10, 7), loc=(5, 3), rowspan=5, colspan=2)

    # Seaborn styling
    sns.set_theme(style="ticks")
    colors = ["#D81B60", "#1E88E5", "#FFC107", "#004D40"] # colorblind friendly
    sns.set_palette(sns.color_palette(colors))
    
    # Log-log plot?
    if log_axes is True:
        ax[0].set(xscale="log", yscale="log")
    else:
        # Plot the unity line (only if log_axes = False)
        if show_unity_line is True:
            # Axis limits for scatterplot (to avoid that the unity line changes the axis limits)
            xlimits, ylimits = axlim(df, feature_1, feature_2)
            ax[0].set(xlim = xlimits, ylim = ylimits)
            ax[0].axline((0,0), slope = 1, linewidth = 1.5, color = "grey", dashes = [6,3])

            
    # Scatter plot
    g1 = sns.scatterplot(ax = ax[0], data = df[df["center_img"]==0],
                         x = feature_1, y = feature_2, hue = "cell_type",hue_order = class_names,
                         alpha = .5)

    # Highlight the points that correspond to the example image
    g2 = sns.scatterplot(ax = ax[0], data = df[df["center_img"]==1],
                         x = feature_1, y = feature_2, hue = "cell_type",hue_order = class_names,
                         style = "center_img", markers = "X", s = 150, edgecolor = "black",
                         legend = False)


    # Plot the example images
    for i, class_name in enumerate(class_names):
        i += 1 # iterate over axes indices, 0 = scatterplot

        img_id = df.loc[(df["cell_type"]==class_name) & (df["center_img"]==1), "id"].values[0]
        img    = skimage.io.imread(fname=str(parent_dir) + "/data/data_original/train/" + img_id + ".png")

        ax[i].imshow(img, cmap = "gray")
        ax[i].set_xticks([])
        ax[i].set_yticks([])
        ax[i].title.set_text(class_name)

        plt.tight_layout()
        plt.savefig("figures/typical_img_" + feature_1 + "_" + feature_2 + ".png", dpi=300)



def scatterplot_and_img(df, class_names, feature_1, feature_2, show_unity_line = False, log_axes = False):
    df = get_prototypical_id(df, class_names, feature_1, feature_2)
    typical_img(df, feature_1, feature_2, show_unity_line = show_unity_line, log_axes = log_axes)
    
    